In [ ]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, recall_score, f1_score, confusion_matrix, \
    precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier


In [ ]:
path = './dados/dados_airbnb'
all_files = glob.glob(path + "/*.csv")
testes = []
treinamento = []
datasets = []
for filename in all_files:
    df = pd.read_csv(filename)
    df['room_type'] = [1 if i == 'Private room' else 0.5 if i == 'Entire home/apt' else 0 for i in df['room_type']]
    df['is_weekend'] = ['weekends' in filename] * len(df)
    df['city'] = filename.split('/')[-1].split('\\')[-1].split('_')[0].upper()
    tr, te = train_test_split(df)

    treinamento.append(tr)
    testes.append(te)

    datasets.append(df)


In [ ]:
# explorando os dados verificamos que existe uma corelação entre a limpeza do local e se o host é super host. quanto mais limpo, maior a nota, superhost (grande parte das vezes) significa uma média maior de nota
index = 16
plt.scatter(datasets[index]['realSum'], datasets[index]['guest_satisfaction_overall'])
plt.title(datasets[index]['city'][0] + ' ' + str(datasets[4]['is_weekend'][0]))

In [ ]:
todos_treinamento = pd.concat(treinamento, ignore_index=True)
todos = pd.concat(datasets, ignore_index=True)
todos_testes = pd.concat(testes, ignore_index=True)

In [ ]:
def separar(dados, y_col = ['guest_satisfaction_overall'], n_x_col = ['city', 'Unnamed: 0', 'attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm',]):

   x_col = [col for col in dados if col not in y_col and col not in n_x_col]
   x = []
   y = []
   for i in range(len(dados)):
       x.append([dados[c][i] for c in x_col])
       y.append([dados[c][i] for c in y_col])
   return x,y

In [ ]:
def testar(modelo, x):
    predito = []
    for i in range(len(x)):
        p = modelo.predict([x[i]])
        predito.append(p)
    return predito
def resultados_numericos(lista):
    return [i[-1][0] for i in lista]
def estrelas(sats):
    return [(i/20).__ceil__() for i in sats ]

In [ ]:
x_treinamento, y_treinamento = separar(todos_treinamento)
x_testes, y_testes = separar(todos_testes)


In [ ]:
m = LinearRegression()
m.fit(x_treinamento, y_treinamento)

In [ ]:
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = [i[-1] for i in y_testes] #resultados_numericos(y_testes)

estrelas_preditas = estrelas(preditos)
estrelas_corretas = estrelas(testes_corretos)

In [ ]:
r2_r = r2_score(estrelas_corretas, estrelas_preditas)
rsme_r = mean_squared_error(estrelas_corretas, estrelas_preditas)
ac_r = accuracy_score(estrelas_corretas, estrelas_preditas)
print(r2_r, rsme_r, ac_r)

In [ ]:
m = (KNeighborsClassifier())
m.fit(x_treinamento, np.array(y_treinamento).ravel())

In [ ]:
y_predito = testar(m, x_testes)
preditos = [i[-1] for i in y_predito] # resultados_numericos(y_predito)
testes_corretos = [i[-1] for i in y_testes] #resultados_numericos(y_testes)
estrelas_preditas = estrelas(preditos)
estrelas_corretas = estrelas(testes_corretos)

In [ ]:
ac_k = accuracy_score(estrelas_corretas, estrelas_preditas)
p_k = precision_score(estrelas_corretas, estrelas_preditas, average="micro")
r_k = recall_score(estrelas_corretas, estrelas_preditas, average="micro")
f1_k = f1_score(estrelas_corretas, estrelas_preditas, average="micro")
menan_k = mean_squared_error(estrelas_corretas, estrelas_preditas)
print(ac_k, p_k, r_k, f1_k, menan_k)

In [ ]:
m = (RandomForestClassifier())
m.fit(x_treinamento, np.array(y_treinamento).ravel())

In [ ]:
y_predito = testar(m, x_testes)
preditos = [i[-1] for i in y_predito] # resultados_numericos(y_predito)
testes_corretos = [i[-1] for i in y_testes] #resultados_numericos(y_testes)
estrelas_preditas = estrelas(preditos)
estrelas_corretas = estrelas(testes_corretos)

In [ ]:
ac_t = accuracy_score(estrelas_corretas, estrelas_preditas)
p_t = precision_score(estrelas_corretas, estrelas_preditas, average="micro")
r_t = recall_score(estrelas_corretas, estrelas_preditas, average="micro")
f1_t = f1_score(estrelas_corretas, estrelas_preditas, average="micro")
menan_t = mean_squared_error(estrelas_corretas, estrelas_preditas)
print(ac_t, p_t, r_t, f1_t, menan_t)